In this notebook, I use a fictional dataset to demonstrate how to compare different tables of assets. This is also a common data manipulation problem in Audit Analytics projects. Financial institutions normally have different sets of coding system for asset positions: for example, for modelling purpose, they have a coding system applicable for each and every asset position, this is shown as column "M_code" in the table "asset_model in the notebook; for financial reporting purpose, they use a different coding system, this is shown as column "balance_code" in the table "asset_balance" in the notebook. In an audit project, we get the source data from the modelling system and mapping rules between different asset codings, with which we have to verify whether the asset balance published by the commpany is correct. In this notebook, I will show you how to compare differnt tables using mapping rules.

In [3]:
import pandas as pd

# 1. Import the three datasets ("balance", "model_balance" and "mapping") from Excel Sheets

import the dataset "balance" as a dataframe

In [4]:
asset_balance=pd.read_excel("Balance_tables.xlsx",sheet_name="balance")
asset_balance

,balance_code,Book_Value,Market_value
0,T.I,31338673,33845766.84
1,T.II.1,69406118,71488301.54
2,T.II.2,59713789,65685167.90
3,T.II.3,58913911,65394441.21
4,T.III.1,29532300,32780853.00
5,T.III.2,31357668,31984821.36
6,T.III.3,66648178,67981141.56
7,T.III.4,65960746,68599175.84
8,T.III.5,45946374,48703156.44
9,T.III.6,6731281,7471721.91


import the dataset "model_balance" and create a pivot table out of it using M_code as index

In [5]:
asset_model=pd.read_excel("Balance_tables.xlsx",sheet_name="model_balance")
asset_model

,M_code,Unnamed: 1,Book_Value,Market_value
0,1,a,18803203.80,2.030746e+07
1,1,b,12535469.20,1.353831e+07
2,2,a,27762447.20,2.859532e+07
3,2,b,27762447.20,2.859532e+07
4,2,c,13881223.60,1.429766e+07
5,3,a,17914136.70,6.568517e+07
6,3,b,41799652.30,4.597962e+07
7,4,a,53022519.90,5.885500e+07
8,4,b,5891391.10,6.539444e+06
9,5,a,17719380.00,1.966851e+07


create a pivot table along the index "M_code".

In [6]:

asset_model=asset_model.pivot_table(index="M_code",values=["Book_Value","Market_value"],aggfunc=sum)
asset_model.reset_index(inplace=True)
asset_model

,M_code,Book_Value,Market_value
0,1,31338673.0,3.384577e+07
1,2,69406118.0,7.148830e+07
2,3,59713789.0,1.116648e+08
3,4,58913911.0,6.539444e+07
4,5,29532300.0,3.278085e+07
5,6,31357668.0,3.198482e+07
6,7,62648178.0,6.798114e+07
7,8,65960746.0,6.859918e+07
8,9,45946374.0,4.870316e+07
9,10,6731281.0,7.471722e+06


For auditing purpose, the goal is to compare the two tables "asset_model" and "asset_balance" by each and every asset code. The table "asset_model" is from the modelling database, therefore has it own coding system, differnt from the coding system used in the table "asset_balance". There is a fixed mapping rules for the two sets of asset coding systems. we import that mapping file and use it to combine the two tables.


In [7]:
mapping=pd.read_excel("Balance_tables.xlsx",sheet_name="mapping")

Create two dictionaries for mapping rules, and use each mapping rule to create a new column of coding in each of the two tables of balances and set the two coding columns as index.

In [8]:

mapping1=dict(mapping[["M_code","balance_code"]].values)
mapping1

{1: 'T.I',
 2: 'T.II.1',
 3: 'T.II.2',
 4: 'T.II.3',
 5: 'T.III.1',
 6: 'T.III.2',
 7: 'T.III.3',
 8: 'T.III.4',
 9: 'T.III.5',
 10: 'T.III.6',
 11: 'T.III.7',
 12: 'T.III.8',
 13: 'T.IV.1',
 14: 'T.IV.2'}

In [9]:
asset_model.loc[:,"balance_code"]=asset_model["M_code"].map(mapping1)
asset_model.set_index(["balance_code","M_code"],inplace=True)
asset_model

,,Book_Value,Market_value
balance_code,M_code,,
T.I,1,31338673.0,3.384577e+07
T.II.1,2,69406118.0,7.148830e+07
T.II.2,3,59713789.0,1.116648e+08
T.II.3,4,58913911.0,6.539444e+07
T.III.1,5,29532300.0,3.278085e+07
T.III.2,6,31357668.0,3.198482e+07
T.III.3,7,62648178.0,6.798114e+07
T.III.4,8,65960746.0,6.859918e+07
T.III.5,9,45946374.0,4.870316e+07


In [10]:
mapping2=dict(mapping[["balance_code","M_code"]].values)
mapping2

{'T.I': 1,
 'T.II.1': 2,
 'T.II.2': 3,
 'T.II.3': 4,
 'T.III.1': 5,
 'T.III.2': 6,
 'T.III.3': 7,
 'T.III.4': 8,
 'T.III.5': 9,
 'T.III.6': 10,
 'T.III.7': 11,
 'T.III.8': 12,
 'T.IV.1': 13,
 'T.IV.2': 14}

In [11]:
asset_balance.loc[:,"M_code"]=asset_balance["balance_code"].map(mapping2)
asset_balance.set_index(["balance_code","M_code"],inplace=True)
asset_balance

,,Book_Value,Market_value
balance_code,M_code,,
T.I,1,31338673,33845766.84
T.II.1,2,69406118,71488301.54
T.II.2,3,59713789,65685167.90
T.II.3,4,58913911,65394441.21
T.III.1,5,29532300,32780853.00
T.III.2,6,31357668,31984821.36
T.III.3,7,66648178,67981141.56
T.III.4,8,65960746,68599175.84
T.III.5,9,45946374,48703156.44


# 2. Join the the two tables along the columns, each row will be aligned by the index.

In [12]:
asset_comparison=pd.concat([asset_balance,asset_model],keys=["Asset Balance","Asset Model"], axis=1)
asset_comparison

Asset Balance              Asset Model              
                       Book_Value Market_value  Book_Value  Market_value
balance_code M_code                                                     
T.I          1           31338673  33845766.84  31338673.0  3.384577e+07
T.II.1       2           69406118  71488301.54  69406118.0  7.148830e+07
T.II.2       3           59713789  65685167.90  59713789.0  1.116648e+08
T.II.3       4           58913911  65394441.21  58913911.0  6.539444e+07
T.III.1      5           29532300  32780853.00  29532300.0  3.278085e+07
T.III.2      6           31357668  31984821.36  31357668.0  3.198482e+07
T.III.3      7           66648178  67981141.56  62648178.0  6.798114e+07
T.III.4      8           65960746  68599175.84  65960746.0  6.859918e+07
T.III.5      9           45946374  48703156.44  45946374.0  4.870316e+07
T.III.6      10           6731281   7471721.91   6731281.0  7.471722e+06
T.III.7      11          71704871  78158309.39  71704871.0  7.815831e+07
T.III.8      12          78073846  85100492.14  78173836.0  8.510049e+07
T.IV.1       13          50401851  51913906.53  50401851.0  5.191391e+07
T.IV.2       14          53373754  59778604.48  53373754.0  5.977860e+07

The joined table "asset_comparison" will be used as the basis table for our comparison.

# 3. Calculate the difference of the two tables by each asset code.

In [13]:
asset_comparison=asset_comparison.stack()
asset_comparison

Asset Balance   Asset Model
balance_code M_code                                          
T.I          1      Book_Value      31338673.00  3.133867e+07
                    Market_value    33845766.84  3.384577e+07
T.II.1       2      Book_Value      69406118.00  6.940612e+07
                    Market_value    71488301.54  7.148830e+07
T.II.2       3      Book_Value      59713789.00  5.971379e+07
                    Market_value    65685167.90  1.116648e+08
T.II.3       4      Book_Value      58913911.00  5.891391e+07
                    Market_value    65394441.21  6.539444e+07
T.III.1      5      Book_Value      29532300.00  2.953230e+07
                    Market_value    32780853.00  3.278085e+07
T.III.2      6      Book_Value      31357668.00  3.135767e+07
                    Market_value    31984821.36  3.198482e+07
T.III.3      7      Book_Value      66648178.00  6.264818e+07
                    Market_value    67981141.56  6.798114e+07
T.III.4      8      Book_Value      65960746.00  6.596075e+07
                    Market_value    68599175.84  6.859918e+07
T.III.5      9      Book_Value      45946374.00  4.594637e+07
                    Market_value    48703156.44  4.870316e+07
T.III.6      10     Book_Value       6731281.00  6.731281e+06
                    Market_value     7471721.91  7.471722e+06
T.III.7      11     Book_Value      71704871.00  7.170487e+07
                    Market_value    78158309.39  7.815831e+07
T.III.8      12     Book_Value      78073846.00  7.817384e+07
                    Market_value    85100492.14  8.510049e+07
T.IV.1       13     Book_Value      50401851.00  5.040185e+07
                    Market_value    51913906.53  5.191391e+07
T.IV.2       14     Book_Value      53373754.00  5.337375e+07
                    Market_value    59778604.48  5.977860e+07

In [14]:
asset_comparison.loc[:,"Asset Balance - Asset Model"]=asset_comparison["Asset Balance"]-asset_comparison["Asset Model"]
asset_audit=asset_comparison.unstack()
asset_audit

Asset Balance              Asset Model                \
                       Book_Value Market_value  Book_Value  Market_value   
balance_code M_code                                                        
T.I          1         31338673.0  33845766.84  31338673.0  3.384577e+07   
T.II.1       2         69406118.0  71488301.54  69406118.0  7.148830e+07   
T.II.2       3         59713789.0  65685167.90  59713789.0  1.116648e+08   
T.II.3       4         58913911.0  65394441.21  58913911.0  6.539444e+07   
T.III.1      5         29532300.0  32780853.00  29532300.0  3.278085e+07   
T.III.2      6         31357668.0  31984821.36  31357668.0  3.198482e+07   
T.III.3      7         66648178.0  67981141.56  62648178.0  6.798114e+07   
T.III.4      8         65960746.0  68599175.84  65960746.0  6.859918e+07   
T.III.5      9         45946374.0  48703156.44  45946374.0  4.870316e+07   
T.III.6      10         6731281.0   7471721.91   6731281.0  7.471722e+06   
T.III.7      11        71704871.0  78158309.39  71704871.0  7.815831e+07   
T.III.8      12        78073846.0  85100492.14  78173836.0  8.510049e+07   
T.IV.1       13        50401851.0  51913906.53  50401851.0  5.191391e+07   
T.IV.2       14        53373754.0  59778604.48  53373754.0  5.977860e+07   

                    Asset Balance - Asset Model                
                                     Book_Value  Market_value  
balance_code M_code                                            
T.I          1                     0.000000e+00  0.000000e+00  
T.II.1       2                     0.000000e+00  0.000000e+00  
T.II.2       3                     0.000000e+00 -4.597962e+07  
T.II.3       4                     0.000000e+00  0.000000e+00  
T.III.1      5                     0.000000e+00  0.000000e+00  
T.III.2      6                     0.000000e+00  0.000000e+00  
T.III.3      7                     4.000000e+06  0.000000e+00  
T.III.4      8                     0.000000e+00  0.000000e+00  
T.III.5      9                     0.000000e+00  0.000000e+00  
T.III.6      10                    0.000000e+00  0.000000e+00  
T.III.7      11                    0.000000e+00  0.000000e+00  
T.III.8      12                   -9.999000e+04  0.000000e+00  
T.IV.1       13                    7.450581e-09  7.450581e-09  
T.IV.2       14                    0.000000e+00  0.000000e+00

In [15]:
asset_audit.to_excel("auditing_results.xlsx")

Now you can see from the table above, there are three errors identified: the first one lies at "Book_Value" of asset position "T.III.3", the second one lies in "Market_value" of asset position "T.II.2"; the last one lies in "Book_Value" of asset position "T.III.8".